# Links

Get direct link(s) to publisher's web page for articles (where available).

In [1]:
import requests

import lxml.html

from time import sleep

In [48]:
MAX_TRIES = 10

def url_for_form(form):

    params = {x.attrib['name']: x.attrib['value'] for x in form.xpath('.//input[@type="hidden"]') if x.name not in ['multilanguage_language_code']}
    
    for try_n in range(1, MAX_TRIES+1):
        
        response = requests.get('http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/core/sfxresolver.cgi', params=params)
                           
        if response.status_code == 200: break

        logging.warn("Network problem getting publisher page (status code {}) on try {}".format(response.status_code, try_n))

        if try_n < MAX_TRIES: sleep(try_n)
            
    else:
        
        logging.error("Out of tries (max {})".format(MAX_TRIES))
        
        return None
    
    url = response.url

    return url

def links_for_pmid(pmid):
    
    if np.isnan(pmid): return None
    
    pmid = int(pmid)
    
    logging.info("Starting PMID '{}'...".format(pmid))
        
    for try_n in range(1, MAX_TRIES+1):
        
        response = requests.get("http://wtgcsfx.hosted.exlibrisgroup.com/wtsc?sid=Entrez:PubMed&id=pmid:{}".format(pmid))
                           
        if response.status_code == 200: break

        logging.warn("Network problem getting WTGC library page (status code {}) on try {}".format(response.status_code, try_n))

        if try_n < MAX_TRIES: sleep(try_n)
            
    else:
        
        logging.error("Out of tries (max {})".format(MAX_TRIES))
        
        return None
        
    dom = lxml.html.fromstring(response.text)
    
    forms = dom.xpath('//form[contains(@name, "basic")]')
    
    urls = [url_for_form(x) for x in forms]

    html = '<ul>' + ' '.join('<li> <a target="_blank" href="{}">Link_{}</a>'.format(url, n) for n, url in enumerate(urls, 1)) + '</ul>'
    
    logging.info("... finished PMID '{}'.".format(pmid))
    
    return html

In [3]:
docs = pd.read_pickle('docs_0.pkl')

docs.shape

(92, 5)

In [4]:
docs['pubmed_id'].dropna().size

81

In [6]:
pubmed_ids = docs[['pubmed_id']].dropna().set_index('pubmed_id', drop=False)['pubmed_id']

In [49]:
links = pubmed_ids.apply(links_for_pmid).to_frame('links')

[2015/Jun/16 12:10:06 INFO    ] Starting PMID '3023614'...
[2015/Jun/16 12:10:10 INFO    ] ... finished PMID '3023614'.
[2015/Jun/16 12:10:10 INFO    ] Starting PMID '2435903'...
[2015/Jun/16 12:10:13 INFO    ] ... finished PMID '2435903'.
[2015/Jun/16 12:10:13 INFO    ] Starting PMID '2435904'...
[2015/Jun/16 12:10:16 INFO    ] ... finished PMID '2435904'.
[2015/Jun/16 12:10:16 INFO    ] Starting PMID '3184128'...
[2015/Jun/16 12:10:19 INFO    ] ... finished PMID '3184128'.
[2015/Jun/16 12:10:19 INFO    ] Starting PMID '3339603'...
[2015/Jun/16 12:10:22 INFO    ] ... finished PMID '3339603'.
[2015/Jun/16 12:10:22 INFO    ] Starting PMID '2892936'...
[2015/Jun/16 12:10:25 INFO    ] ... finished PMID '2892936'.
[2015/Jun/16 12:10:25 INFO    ] Starting PMID '2840498'...
[2015/Jun/16 12:10:28 INFO    ] ... finished PMID '2840498'.
[2015/Jun/16 12:10:28 INFO    ] Starting PMID '2840504'...
[2015/Jun/16 12:10:30 INFO    ] ... finished PMID '2840504'.
[2015/Jun/16 12:10:30 INFO    ] Starting

In [39]:
HTML(links.to_html())

,links
pubmed_id,
3023614,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=3023614&month=12&author=Tamazawa%2C+K&issue=12&ISBN=&article=Stereoselectivity+of+a+potent+calcium+antagonist%2C+1-benzyl-3-pyrrolidinyl+methyl+2%2C6-dimethyl-4-(m-nitrophenyl)-1%2C4-dihydropyridine-3%2C5-dicarboxylate.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&volume=29&recipient=libadmin%40sanger.ac.uk&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&year=1986&pages=2504-11"">Link_2"
2435903,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=2435903&month=4&author=Atwal%2C+K+S&issue=4&ISBN=&article=Synthesis+and+biological+activity+of+novel+calcium+channel+blockers%3A+2%2C5-dihydro-4-methyl-2-phenyl-1%2C5-benzothiazepine-3-carboxylic+acid+esters+and+2%2C5-dihydro-4-methyl-2-phenyl-1%2C5-benzodiazepine-3-carboxylic+acid+esters.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&recipient=libadmin%40sanger.ac.uk&volume=30&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&pages=635-40&year=1987"">Link_2"
2435904,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=2435904&month=4&author=Baldwin%2C+J+J&issue=4&ISBN=&article=Diethyl+3%2C6-dihydro-2%2C4-dimethyl-2%2C6-methano-1%2C3-benzothiazocine-5%2C11-+dicarboxylates+as+calcium+entry+antagonists%3A+new+conformationally+restrained+analogues+of+Hantzsch+1%2C4-dihydropyridines+related+to+nitrendipine+as+probes+for+receptor-site+conformation.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&recipient=libadmin%40sanger.ac.uk&volume=30&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&pages=690-5&year=1987"">Link_2"
3184128,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=3184128&month=11&author=Sanfilippo%2C+P+J&issue=11&ISBN=&article=Synthesis+of+(aryloxy)alkylamines.+2.+Novel+imidazo-fused+heterocycles+with+calcium+channel+blocking+and+local+anesthetic+activity.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&recipient=libadmin%40sanger.ac.uk&volume=31&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&pages=2221-7&year=1988"">Link_2"
3339603,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=3339603&month=2&author=Fossheim%2C+R&issue=2&ISBN=&article=Crystal+structures+and+pharmacologic+activities+of+1%2C4-dihydropyridine+calcium+channel+antagonists+of+the+isobutyl+methyl+2%2C6-dimethyl-4-(substituted+phenyl)-1%2C4-dihydropyridine-3%2C5-dicarboxylate+(nisoldipine)+series.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&volume=31&recipient=libadmin%40sanger.ac.uk&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&year=1988&pages=300-5"">Link_2"
2892936,Link_1 Link_2
2840498,Link_1 Link_2
2840504,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=2840504&month=8&author=Taylor%2C+M+D&issue=8&ISBN=&article=2-(2-Aryl-2-oxoethylidene)-1%2C2%2C3%2C4-tetrahydropyridines.+Novel+isomers+of+1%2C4-dihydropyridine+calcium+channel+blockers.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&recipient=libadmin%40sanger.ac.uk&volume=31&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&pages=1659-64&year=1988"">Link_2"
2552119,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/p

In [40]:
links.to_pickle('links.pkl')